In [1]:
import sys
sys.path.append("/home/jupyter-group3/reconstruction/reconstruction-deep-network")

In [2]:
import numpy as np
import os
import torch
from argparse import ArgumentParser
import yaml
from tqdm import tqdm

import reconstruction_deep_network
from reconstruction_deep_network.trainer.trainer import ModelTrainer
from reconstruction_deep_network.data_loader.custom_loader import CustomDataLoader

In [3]:
module_dir = reconstruction_deep_network.__path__[0]
root_dir = os.path.dirname(module_dir)
data_dir = os.path.join(root_dir, "data", "v1")
scans_dir = os.path.join(data_dir, "scans")
all_scans = [scan for scan in os.listdir(scans_dir) if not scan.endswith('.ipynb_checkpoints')]

In [4]:
def save_image_encoding(img_encoding: np.ndarray, scan_id: str, img_name: str, img_encoding_dir: str):
    dir_name = os.path.join(img_encoding_dir, scan_id)
    if not os.path.isdir(dir_name):
        os.makedirs(dir_name)
    
    file_name = os.path.join(dir_name, f"{img_name}.npz")    
    np.savez(file_name, latent = img_encoding)

In [5]:
def parse_args(args=None):
    parser = ArgumentParser()
    parser.add_argument("--data_path", dest = "data_path", type = str, default = data_dir)
    parser.add_argument("--debug", dest = "debug", type = bool, default = True)
    parser.add_argument("--id", dest = "id", type = str, default = None)
    # parser.add_argument("--file_type", dest = "file_type", type = str)
    # parser.add_argument("--dataset_mode", dest = "dataset_mode", type = str)

    args = parser.parse_args(args)
    return args

In [6]:
all_scans

['8WUmhLawc2A',
 '29hnd4uzFmX',
 'EDJbREhghzL',
 'EU6Fwq7SyZv',
 'D7N2EKCX4Sj',
 'dhjEzFoUFzH',
 'e9zR4mvMWw7',
 '1pXnuDYAj8r',
 'E9uDoFAP3SH',
 'ARNzJeq3xxb',
 '8194nk5LbLH',
 'b8cTxDM8gDG',
 'B6ByNegPMKs',
 '2azQ1b91cZZ',
 '2t7WUuJeko7',
 'fzynW3qQPVF',
 '1LXtFkjw3qL',
 '5ZKStnWn8Zo',
 '5LpN3gDmAk7',
 '17DRP5sb8fy',
 '759xd9YjKW5',
 'ac26ZMwG7aT',
 '7y3sRwLe3Va',
 'D7G3Y4RVNrH',
 'aayBHfsNo7d',
 'cV4RVeZvu5T',
 'GdvgFV5R1Z5',
 '5q7pvUzZiYa',
 '2n8kARJN3HM',
 '82sE5b5pLXE']

In [7]:
def main(args):
    
    main_data_dir = args.data_path
    scans_dir = os.path.join(main_data_dir, "scans")
    img_latents_dir = os.path.join(main_data_dir, "image_latents")
    if not os.path.isdir(img_latents_dir):
        os.makedirs(img_latents_dir)

    # initialize model trainer to encode images
    model_trainer = ModelTrainer()

    house_scans = None
    if args.debug:
        house_scans = ["17DRP5sb8fy"]
    else:
        if args.id == "ALL":
            house_scans = all_scans
        else:
            house_scans = [args.id]
    print(f"Number of house scans: {len(house_scans)}")
    for itr, house_scan in enumerate(house_scans):

        dataset = CustomDataLoader(debug = False)
        dataset.metadata = dataset._limit_dataset(scan_id = house_scan)

        print(f"House Scan: {house_scan}, Number of Scenes: {len(dataset)}")
        if house_scan in os.listdir(img_latents_dir):
            print(f"Already stored {house_scan}...")
            continue

        for scene_idx in range(len(dataset)):

            scene_dict = dataset[scene_idx]
            img_paths = scene_dict["images_paths"]
            path_components = img_paths[0].split("/")
            img_name = path_components[-1].split("_")[0]
            if scene_dict["img_encoding"] is not None: continue

            images = scene_dict["images"]
            images = torch.from_numpy(images)
            images = torch.unsqueeze(images, 0)
            img_encoding = model_trainer.encode_image(images, model_trainer.vae)
            img_encoding = img_encoding.numpy().squeeze()

            save_image_encoding(img_encoding, house_scan, img_name, img_latents_dir)
            print(f"Saved encoding: {house_scan}, scene: {img_name}")

In [8]:
args = parse_args(["--data_path", data_dir, "--debug", False, "--id", "ALL"])

In [9]:
main(args)

Loading image encoders...
Number of house scans: 30
House Scan: 8WUmhLawc2A, Number of Scenes: 115
Already stored 8WUmhLawc2A...
House Scan: 29hnd4uzFmX, Number of Scenes: 143
Already stored 29hnd4uzFmX...
House Scan: EDJbREhghzL, Number of Scenes: 71
Already stored EDJbREhghzL...
House Scan: EU6Fwq7SyZv, Number of Scenes: 78
Already stored EU6Fwq7SyZv...
House Scan: D7N2EKCX4Sj, Number of Scenes: 267
Already stored D7N2EKCX4Sj...
House Scan: dhjEzFoUFzH, Number of Scenes: 218
Already stored dhjEzFoUFzH...
House Scan: e9zR4mvMWw7, Number of Scenes: 91
Already stored e9zR4mvMWw7...
House Scan: 1pXnuDYAj8r, Number of Scenes: 95
Already stored 1pXnuDYAj8r...
House Scan: E9uDoFAP3SH, Number of Scenes: 217
Already stored E9uDoFAP3SH...
House Scan: ARNzJeq3xxb, Number of Scenes: 98
Already stored ARNzJeq3xxb...
House Scan: 8194nk5LbLH, Number of Scenes: 20
Already stored 8194nk5LbLH...
House Scan: b8cTxDM8gDG, Number of Scenes: 117
Already stored b8cTxDM8gDG...
House Scan: B6ByNegPMKs, Numbe

In [10]:
scans_dir = os.path.join(data_dir, "scans")
os.listdir(scans_dir)

['8WUmhLawc2A',
 '29hnd4uzFmX',
 'EDJbREhghzL',
 'EU6Fwq7SyZv',
 'D7N2EKCX4Sj',
 'dhjEzFoUFzH',
 'e9zR4mvMWw7',
 '1pXnuDYAj8r',
 'E9uDoFAP3SH',
 'ARNzJeq3xxb',
 '8194nk5LbLH',
 'b8cTxDM8gDG',
 'B6ByNegPMKs',
 '2azQ1b91cZZ',
 '2t7WUuJeko7',
 'fzynW3qQPVF',
 '1LXtFkjw3qL',
 '5ZKStnWn8Zo',
 '5LpN3gDmAk7',
 '17DRP5sb8fy',
 '.ipynb_checkpoints',
 '759xd9YjKW5',
 'ac26ZMwG7aT',
 '7y3sRwLe3Va',
 'D7G3Y4RVNrH',
 'aayBHfsNo7d',
 'cV4RVeZvu5T',
 'GdvgFV5R1Z5',
 '5q7pvUzZiYa',
 '2n8kARJN3HM',
 '82sE5b5pLXE']